In [1]:
import math

In [46]:
class Word:
    def __init__(self, wordLength, ram, x):
        self.wordLength = wordLength
        self.machine = ram.machine
        self.ram = ram
        self.set(x)
        
    def checkError(self, v):
        if v > self.ram.maxValue or v < 0:
            print(f"Error! Tried to set value of {v} on the memory with {self.wordLength}-bit word length.")
            assert(False)
    
    def set(self, x):
        self.checkError(x)
        self.x = x
    
    def lg(self):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, int(math.log(self.x, 2)))
    
    def __add__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x + rhs.x)
    def __sub__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x - rhs.x)
    def  __mul__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x * rhs.x)
    def __truediv__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x // rhs.x)
    def __and__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x & rhs.x)
    def __or__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x | rhs.x)
    def __xor__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x ^ rhs.x)
    def __lshift__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x << rhs.x)
    def __rshift__(self, rhs):
        self.machine.totalExecution += 1
        return Word(self.wordLength, self.ram, self.x >> rhs.x)

In [29]:
class WordRAM:
    def __init__(self, size, wordLength, machine):
        self.size = size
        self.wordLength = wordLength
        self.machine = machine
        
        self.maxValue = 2 ** wordLength - 1
        
        self._d = [Word(self.wordLength, self, 0) for _ in range(self.size)]
    
    def get(self, i):
        self.machine.totalExecution += 1
        return self._d[i]
    
    def set(self, i, w):
        assert(isinstance(w, Word))
        self.machine.totalExecution += 1
        self._d[i] = w

In [30]:
class WordRAMMachine:
    def __init__(self, wordLength):
        self.wordLength = wordLength
        
        self.totalMemSize = 0
        self.totalExecution = 0
        
    def allocateMem(self, size, wordLength):
        if (wordLength > self.wordLength):
            print(f"Error! Tried to allocate memory with word length of {wordLength} bits on the machine with {self.wordLength}-bit word length.")
            return None
        mem = WordRAM(size, wordLength, self)
        self.totalMemSize += size * wordLength
        return mem